### Definición del Problema

Sudoku es un puzzle numérico popular para todas las personas y puede ser modelado como un problema de programación lineal, usando variables binarias

La meta (o objetivo principal) del Sudoku es colocar los dígitos del 1 al 9 en un tablero de $9 \times 9$, donde algunas celdas de este tablero ya tienen fijado un dígito. La solución debe seguir las siguientes reglas:
- Los números de 1 al 9 deben estar en cada sub-tablero de $3 \times 3$
- Cada columna debe contener los números del 1 al 9
- Cada fila debe contener los números del 1 al 9

In [13]:
table = [
  [0,0,0, 0,0,0, 0,2,9],
  [0,0,0, 0,7,8, 0,0,5],
  [0,0,0, 1,9,0, 8,3,0],

  [0,6,1, 0,3,9, 2,5,0],
  [0,3,0, 0,0,0, 0,9,0],
  [0,8,9, 4,6,0, 3,7,0],
  
  [0,5,2, 0,1,4, 0,0,0],
  [1,0,0, 3,5,0, 0,0,0],
  [3,7,0, 0,0,0, 0,0,0],
]
G = [ (i+1,j+1,table[i][j]) for i in range(9) for j in range(9) ]

In [14]:
from pulp import *

P = LpProblem("Sudoku-Problem", LpMinimize)

In [15]:
VALS=ROWS=COLS = range(1, 10)
BOXES = [ [(3 *i + k+1, 3*j + l+1) for k in range(3) for l in range(3) ] for i in range(3) for j in range(3) ]

Las variables decisión del problema son:
$$x_{ijk} = \bigg\{ \begin{matrix}
1 & \text{celda }(i,j) = k\\
0 & \text{e.o.c}
\end{matrix}$$

In [16]:
x = LpVariable.dicts("Variable", (ROWS,COLS,VALS), cat="Binary")

La función objetivo se vuelve irrelevante porque cada punto que satisface las restricciones representará una solución del puzzle
$$\begin{matrix}
\min 0^T x_{ijk} & \forall i,j,k
\end{matrix}$$

In [17]:
P += 0*lpSum( [x[i][j][k] for i in ROWS for j in COLS for k in VALS] )


Entonces, tenemos las siguientes restricciones:

Cada fila contiene exactamente un numero entero del 1 al 9
$$\begin{matrix}
\displaystyle\sum_{i=1}^9 x_{ijk} = 1 & \forall j,k
\end{matrix}$$

In [18]:
for k in VALS:
  for j in COLS:
    P += lpSum([x[i][j][k] for i in ROWS]) == 1

Cada columna contiene exactamente un número entero del 1 al 9
$$\begin{matrix}
\displaystyle\sum_{j=1}^9 x_{ijk} = 1 & \forall i,k
\end{matrix}$$

In [19]:
for k in VALS:
  for i in ROWS:
    P += lpSum([x[i][j][k] for j in COLS]) == 1

Cada celda tiene asignado solamente un numero entero del 1 al 9
$$\begin{matrix}
\displaystyle\sum_{k=1}^9 x_{ijk} = 1 & \forall i,j
\end{matrix}$$

In [20]:
for i in ROWS:
  for j in COLS:
    P += lpSum([x[i][j][k] for k in VALS]) == 1

Cada sub-tabla o sub-tablero contiene exactamente todos los números del 1 al 9
$$\begin{matrix}
\displaystyle\sum_{i=3p-2}^{3p} \sum_{j=3q-2}^{3q} x_{ijk} = 1 & \forall k \land \forall p,q \in \{ 1,2,3 \}
\end{matrix}$$

In [21]:
for k in VALS:
  for box in BOXES:
    P += lpSum( [x[i][j][k] for (i,j) in box] ) == 1


In [22]:
for (i,j,k) in G:
  if k > 0: P += x[i][j][k] == 1 

Se debe tener en cuenta que, en la gran mayoría de las tablas, existe una solución única, pero puede existir más de una solución por tabla, e incluso, no tener solución

In [23]:
P.solve()
LpStatus[P.status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/zkorpion/Personal Folders/Projects/python-vault/env/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b9afc17fce2b4d2faf4820547e1376c5-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/b9afc17fce2b4d2faf4820547e1376c5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 361 COLUMNS
At line 4769 RHS
At line 5126 BOUNDS
At line 5857 ENDATA
Problem MODEL has 356 rows, 730 columns and 2948 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 second

'Optimal'

In [24]:
for v in P.variables():
  if v.varValue:
    print( f"{v.name} = {v.varValue}" )

Variable_1_1_8 = 1.0
Variable_1_2_1 = 1.0
Variable_1_3_7 = 1.0
Variable_1_4_5 = 1.0
Variable_1_5_4 = 1.0
Variable_1_6_3 = 1.0
Variable_1_7_6 = 1.0
Variable_1_8_2 = 1.0
Variable_1_9_9 = 1.0
Variable_2_1_6 = 1.0
Variable_2_2_9 = 1.0
Variable_2_3_3 = 1.0
Variable_2_4_2 = 1.0
Variable_2_5_7 = 1.0
Variable_2_6_8 = 1.0
Variable_2_7_1 = 1.0
Variable_2_8_4 = 1.0
Variable_2_9_5 = 1.0
Variable_3_1_5 = 1.0
Variable_3_2_2 = 1.0
Variable_3_3_4 = 1.0
Variable_3_4_1 = 1.0
Variable_3_5_9 = 1.0
Variable_3_6_6 = 1.0
Variable_3_7_8 = 1.0
Variable_3_8_3 = 1.0
Variable_3_9_7 = 1.0
Variable_4_1_4 = 1.0
Variable_4_2_6 = 1.0
Variable_4_3_1 = 1.0
Variable_4_4_7 = 1.0
Variable_4_5_3 = 1.0
Variable_4_6_9 = 1.0
Variable_4_7_2 = 1.0
Variable_4_8_5 = 1.0
Variable_4_9_8 = 1.0
Variable_5_1_7 = 1.0
Variable_5_2_3 = 1.0
Variable_5_3_5 = 1.0
Variable_5_4_8 = 1.0
Variable_5_5_2 = 1.0
Variable_5_6_1 = 1.0
Variable_5_7_4 = 1.0
Variable_5_8_9 = 1.0
Variable_5_9_6 = 1.0
Variable_6_1_2 = 1.0
Variable_6_2_8 = 1.0
Variable_6_3_